In [1]:
from numbers import Number
import numpy as np

In [2]:
HANDLED_FUNCTIONS = {}


class DiagonalArray(np.lib.mixins.NDArrayOperatorsMixin):
    def __init__(self, N, value):
        self._N = N
        self._i = value

    def __repr__(self):
        return f"{self.__class__.__name__}(N={self._N}, value={self._i})"

    def __array__(self, dtype=None, copy=None):
        if copy is False:
            raise ValueError("`copy=False` isn't supported. A copy is always created.")
        return self._i * np.eye(self._N, dtype=dtype)

    def __array_ufunc__(self, ufunc, method, *inputs, **kwargs):
        if method == "__call__":
            N = None
            scalars = []
            for input in inputs:
                # In this case we accept only scalar numbers or DiagonalArrays.
                if isinstance(input, Number):
                    scalars.append(input)
                elif isinstance(input, self.__class__):
                    scalars.append(input._i)
                    if N is not None:
                        if N != input._N:
                            raise TypeError("inconsistent sizes")
                    else:
                        N = input._N
                else:
                    return NotImplemented
            return self.__class__(N, ufunc(*scalars, **kwargs))
        else:
            return NotImplemented

    def __array_function__(self, func, types, args, kwargs):
        print("meow")
        if func not in HANDLED_FUNCTIONS:
            return NotImplemented
        # Note: this allows subclasses that don't override
        # __array_function__ to handle DiagonalArray objects.
        if not all(issubclass(t, self.__class__) for t in types):
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)

In [3]:
arr = DiagonalArray(5, 1)

In [4]:
arr + 3

DiagonalArray(N=5, value=4)

In [5]:
arr > 0

DiagonalArray(N=5, value=True)

In [6]:
np.concatenate([arr, arr])

meow


TypeError: no implementation found for 'numpy.concatenate' on types that implement __array_function__: [<class '__main__.DiagonalArray'>]